## Config

In [1]:
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [ ]:
from numpy import outer, eye, ones, zeros, diag, log, sqrt, exp, pi
from numpy.linalg import inv, solve
from numpy.random import multivariate_normal as mvnormal, normal, gamma, beta, binomial
from scipy.special import gammaln
from scipy.stats import norm

import numpy as np

## EM-algorithm

In [3]:
# Aalto University, School of Science
# T-61.5140 Machine Learning: Advanced probabilistic Methods
# Author: antti.kangasraasio@aalto.fi, 2016

import copy

class EM_algo():
    """
        A superclass for different EM-fitted models.
    """

    def __init__(self, hyperparams, X=None, Y=None, ndata=0, pdata=0):
        """
            Initialize model based either on given data (X, Y) or
            on given data dimensionality (ndata, pdata).
        """
        if X != None and Y != None:
            self.X = X
            self.Y = Y
            self.ndata = len(self.X)
            self.pdata = len(self.X[0])
        if ndata and pdata:
            self.X = None
            self.Y = None
            self.ndata = ndata
            self.pdata = pdata
        self.h = hyperparams
        self.p = dict() # model parameters
        self.reset()
        if X != None and Y != None:
            self.current_logl, self.cll = self.logl()


    def reset(self):
        """
            Reset priors and draw parameter estimates from prior.
        """
        raise NotImplementedError("Subclass implements")


    def draw(self, item):
        """
            Draw a data sample from the current predictive distribution.
            Returns the drawn y and z-values.
        """
        raise NotImplementedError("Subclass implements")


    def logl(self):
        """
            Calculates the full log likelihood for this model.
            Returns the logl (and the values of each term for debugging purposes)
        """
        raise NotImplementedError("Subclass implements")


    def EM_iter(self):
        """
            Executes a single round of EM updates for this model.
        """
        raise NotImplementedError("Subclass implements")


    def EM_fit(self, alim=1e-10, maxit=1e4):
        """
            Calls the EM_iter repeatedly until the log likelihood
            of the model increases less than 'alim' in absolute
            value or after 'maxit' iterations have been done.

            Returns the number of EM-iterations, final log likelihood
            value and a string that explains the end condition.
        """
        logl, ll = self.logl()
        for i in range(int(maxit)):
            self.EM_iter()
            logl2, ll2 = self.logl()
            adiff = abs(logl2 - logl)
            if adiff < alim:
                return i+1, logl2, "alim"
            logl = logl2
        return maxit, logl2, "maxit"


    def assert_logl_increased(self, event):
        """
            Checks that the log likelihood increased since model
            initialization or the time this function was last called.
        """
        newlogl, ll = self.logl()
        if self.current_logl - newlogl > 1e-3:
            self.debug_logl(self.cll, ll)
            raise ValueError("logl increased after %s" % (event))
        self.current_logl, self.cll = newlogl, ll


    def get_p(self):
        """
            Returns a copy of the model parameters.
        """
        return copy.deepcopy(self.p)


    def set_p(self, p):
        """
            Sets the model parameters.
        """
        self.p = p.copy()


    def print_p(self):
        """
            Prints the model parameters, one at each line.
        """
        for k, v in self.p.items():
            print("%s = %s" % (k, v))


    def pretty_vector(self, x):
        """
            Returns a formatted version of a vector.
        """
        s = ["("]
        s.extend(["%.2f, " % (xi) for xi in x[:-1]])
        s.append("%.2f)" % (x[-1]))
        return "".join(s)


    def debug_logl(self, ll1, ll2):
        """
            Prints an analysis of the per-term change in
            log likelihood from ll1 to ll2.
        """
        print("Logl      before     after")
        for v1, v2, i in zip(ll1, ll2, range(len(ll1))):
            if v1 > v2:
                d = ">"
            elif v2 > v1:
                d = "<"
            else:
                d = "="
            print("Term %02d: %7.3f %s %7.3f" % (i, v1, d, v2))
        print("Total    %7.3f   %7.3f" % (sum(ll1), sum(ll2)))



## Linear model

In [4]:
# Aalto University, School of Science
# T-61.5140 Machine Learning: Advanced probabilistic Methods
# Author: antti.kangasraasio@aalto.fi, 2016

from numpy import outer, eye, ones, zeros, log, sqrt, exp, pi
from numpy.linalg import inv, solve
from numpy.random import multivariate_normal as mvnormal, normal, gamma, beta, binomial
from scipy.special import gammaln

class EM_algo_LM(EM_algo):
    """
        A linear gaussian model.
    """

    def reset(self):
        """
            Reset priors and draw parameter estimates from prior.
        """
        # priors
        self.lbd_phi0       = self.h["lbd_phi0"]
        self.alpha_s20      = self.h["alpha_s20"]
        self.beta_s20       = self.h["beta_s20"]
        self.sigma_phi0     = eye(self.pdata) * self.h["lbd_phi0"]
        self.sigma_phi0_inv = eye(self.pdata) / self.h["lbd_phi0"]
        self.mu_phi0        = ones(self.pdata) * self.h["mu_phi0"]

        # initial parameter estimates drawn from prior
        self.p           = dict()
        self.p["sigma2"] = 1.0 / gamma(self.alpha_s20, 1.0 / self.beta_s20) # inverse gamma
        self.p["phi"]    = mvnormal(self.mu_phi0, self.p["sigma2"] * self.sigma_phi0)


    def draw(self, item):
        """
            Draw a data sample from the current predictive distribution.
            Returns the y-value (and a constant z-value for compatibility)
        """
        mean = float(item.dot(self.p["phi"]))
        std  = sqrt(self.p["sigma2"])
        return normal(mean, std), 1


    def logl(self):
        """
            Calculates the full log likelihood for this model.
            Returns the logl (and the values of each term for debugging purposes)
        """
        ll    = zeros(8)
        phie  = self.p["phi"] - self.mu_phi0
        err   = (self.X.dot(self.p["phi"]) - self.Y) ** 2
        # p(y)
        ll[0] = - 0.5 * log(2 * pi * self.p["sigma2"]) * self.ndata
        ll[1] = sum(- 0.5 * err / self.p["sigma2"])
        # p(phi)
        ll[2] = - 0.5 * log(2 * pi * self.lbd_phi0 * self.p["sigma2"]) * self.pdata
        ll[3] = - 0.5 * phie.T.dot(phie) / (self.lbd_phi0 * self.p["sigma2"])
        # p(sigma2)
        ll[4] = self.alpha_s20 * log(self.beta_s20)
        ll[5] = - gammaln(self.alpha_s20)
        ll[6] = - (self.alpha_s20 + 1.0) * log(self.p["sigma2"])
        ll[7] = - self.beta_s20 / self.p["sigma2"]
        return sum(ll), ll


    def EM_iter(self):
        """
            Executes a single round of EM updates for this model.

            Has checks to make sure that updates increase logl and
            that parameter values stay in sensible limits.
        """
        # phi
        sumxx         = self.X.T.dot(self.X)
        sumxy         = self.X.T.dot(self.Y)
        sigma_mu      = self.sigma_phi0_inv.dot(self.mu_phi0)
        sigma_phi_inv = self.sigma_phi0_inv + sumxx
        self.p["phi"] = solve(sigma_phi_inv, sigma_mu + sumxy)
        self.assert_logl_increased("phi update")

        # sigma2
        phie = (self.p["phi"] - self.mu_phi0) ** 2
        err  = (self.X.dot(self.p["phi"]) - self.Y) ** 2
        num  = self.beta_s20 + 0.5 * sum(err) + 0.5 * sum(phie) / self.lbd_phi0
        den  = self.alpha_s20 + 1.0 + 0.5 * (self.ndata + self.pdata)
        self.p["sigma2"] = num / den
        if self.p["sigma2"] < 0.0:
            raise ValueError("sigma2 < 0.0")
        self.assert_logl_increased("sigma2 update")


    def print_p(self):
        """
            Prints the model parameters, one at each line.
        """
        print("phi    : %s" % (self.pretty_vector(self.p["phi"])))
        print("sigma2 : %.3f" % (self.p["sigma2"]))



## Generator

In [5]:
# Aalto University, School of Science
# T-61.5140 Machine Learning: Advanced probabilistic Methods
# Author: antti.kangasraasio@aalto.fi, 2016

from numpy import zeros
from numpy.random import randn


def generate_X(ndata, pdata):
    """
        Return a matrix of normally distributed random values.
    """
    X = randn(ndata, pdata)
    return X


def generate_YZ(X, distribution):
    """
        Draw observations Y and latent variable values Z from a distribution.
    """
    ndata = len(X)
    Y = zeros(ndata)
    Z = zeros(ndata)
    for i in range(ndata):
        Y[i], Z[i] = distribution.draw(X[i])
    return Y, Z


def get_hyperp():
    """
        Return model hyperparameters.
    """
    return {
            "alpha_s20": 5.0,
            "beta_s20" : 1.0,
            "lbd_phi0" : 1.0,
            "mu_phi0"  : 0.0,
            "alpha_w0" : 3.0,
            "beta_w0"  : 3.0,
            }


## Mixture model

Note that for responsibilities 
$$\gamma_1 =  \frac{ w_S \mathcal{N}(y_t \ | \ x_t \phi_{1_S}, \sigma_{1_S}^2)}{ w_S \mathcal{N}(y_t \ | \ x_t \phi_{1_S} , \sigma_{1_S}^2) \; + \; (1 - w_S) \mathcal{N}(y_t \ | \ x_t \phi_{2_S}, \sigma_{2_S}^2)} $$
$$\gamma_2 =  \frac{ (1 - w_S) \mathcal{N}(y_t \ | \ x_t \phi_{2_S}, \sigma_{2_S}^2)}{ w_S \mathcal{N}(y_t \ | \ x_t \phi_{1_S} , \sigma_{1_S}^2) \; + \; (1 - w_S) \mathcal{N}(y_t \ | \ x_t \phi_{2_S}, \sigma_{2_S}^2)} $$

If we only need to know whether $\gamma_1 > \gamma_2$, then we need to only compute

$$ w_S \mathcal{N}(y_t \ | \ x_t \phi_{1_S}, \sigma_{1_S}^2) > (1 - w_S) \mathcal{N}(y_t \ | \ x_t \phi_{2_S}, \sigma_{2_S}^2)$$

In [ ]:
from numpy import outer, eye, ones, zeros, diag, log, sqrt, exp, pi
from numpy.linalg import inv, solve
from numpy.random import multivariate_normal as mvnormal, normal, gamma, beta, binomial
from scipy.special import gammaln
from scipy.stats import norm
import numpy as np

class EM_algo_MM(EM_algo):
    """
        A mixture of two linear models.
    """

    def reset(self):
        """
            Reset priors and draw parameter estimates from prior.
        """
        # priors
        self.alpha_w0       = self.h["alpha_w0"]
        self.beta_w0        = self.h["beta_w0"]

        # Same priors for phi1 and phi2, s2_1, s2_2, don't bother to copy vars twice
        # i.e. alpha_s2_1_0 = alpha_s2_2_0 = alpha_s20
        self.lbd_phi0       = self.h["lbd_phi0"]
        self.alpha_s20      = self.h["alpha_s20"]
        self.beta_s20       = self.h["beta_s20"]
        self.sigma_phi0     = eye(self.pdata) * self.h["lbd_phi0"]
        self.sigma_phi0_inv = eye(self.pdata) / self.h["lbd_phi0"]
        self.mu_phi0        = ones(self.pdata) * self.h["mu_phi0"]
        
        # Precalculations:
        self.w_gamma_ln_multiplier  = gammaln(self.alpha_w0 + self.beta_w0)
        self.w_gamma_ln_multiplier -= gammaln(self.alpha_w0)
        self.w_gamma_ln_multiplier -= gammaln(self.beta_w0)
        
        
        # initial parameter estimates drawn from prior
        self.p             = dict()
        # Weights
        self.p["w"]        = beta(self.alpha_w0, self.beta_w0)
        # Responsibilities (TODO: do we need this here?)
        self.p["gamma"]    = binomial(1, self.p["w"], self.ndata)
        # Component 1
        self.p["sigma2_1"] = 1.0 / gamma(self.alpha_s20, 1.0 / self.beta_s20) # inverse gamma
        self.p["phi_1"]    = mvnormal(self.mu_phi0, self.p["sigma2_1"] * self.sigma_phi0)
        # Component 2
        self.p["sigma2_2"] = 1.0 / gamma(self.alpha_s20, 1.0 / self.beta_s20) # inverse gamma
        self.p["phi_2"]    = mvnormal(self.mu_phi0, self.p["sigma2_2"] * self.sigma_phi0)


    def draw(self, item):
        """
            Draw a data sample from the current predictive distribution.
            Returns the y-value and z-value
        """    
        mean1 = float(item.dot(self.p["phi_1"]))
        std1  = sqrt(self.p["sigma2_1"])
        mean2 = float(item.dot(self.p["phi_2"]))
        std2  = sqrt(self.p["sigma2_2"])
        
        # Responsibilites for item (item is a data point)
        propto_gamma1 = self.p["w"] * norm.pdf(item, mean1, std1)
        propto_gamma2 = (1 - self.p["w"]) * norm.pdf(item, mean2, std2)
        
        if (propto_gamma1 > propto_gamma2)
            return normal(mean1, std1), 1
        else
            return normal(mean2, std2), 2


    def logl(self):
        """
            Calculates the full log likelihood for this model.
            Returns the logl (and the values of each term for debugging purposes)
        """
        ll         = zeros(20)
        phi_1_diff = self.p["phi_1"] - self.mu_phi0
        phi_2_diff = self.p["phi_2"] - self.mu_phi0
        phi_1_err  = phi_1_diff.T.dot(phi_1_diff)
        phi_2_err  = phi_2_diff.T.dot(phi_2_diff)
        err_1      = (self.X.dot(self.p["phi_1"]) - self.Y) ** 2
        err_2      = (self.X.dot(self.p["phi_2"]) - self.Y) ** 2
        
        
        # Responsibilities
#         mean1 = float(item.dot(self.p["phi_1"]))
#         std1  = sqrt(self.p["sigma2_1"])
#         mean2 = float(item.dot(self.p["phi_2"]))
#         std2  = sqrt(self.p["sigma2_2"])
#         # data is 1 dimensional, and norm.pdf works on a vector returning probability for each separately
#         propto_gamma1 = self.p["w"] * norm.pdf(self.X, mean1, std1)
#         propto_gamma2 = (1 - self.p["w"]) * norm.pdf(self.X, mean2, std2)
#         # elementwise, works because numpy
#         gamma = propto_gamma1 / (propto_gamma1 + propto_gamma2)
        gamma = self.p["gamma"]
        
        ### posterior factorizes p(y,z,w,phi,sigma) = p(y)p(z)p(w)p(phi)p(sigma)
        
        ### p(y)
        ll[0] = self.p["w"]       * (-0.5 * log(2 * pi * self.p["sigma2_1"]) * self.ndata)
        ll[1] = self.p["w"]       * sum(- 0.5 * err_1 / self.p["sigma2_1"])
        ll[2] = (1 - self.p["w"]) * (-0.5 * log(2 * pi * self.p["sigma2_2"]) * self.ndata)
        ll[3] = (1 - self.p["w"]) * sum(- 0.5 * err_2 / self.p["sigma2_2"])
        
        ### p(z)
        ll[4] = (gamma * log(self.p["w"])) + ((1 - gamma) * log(1 - self.p["w"]))
        
        ### p(w)
        ll[5] = self.w_gamma_ln_multiplier
        ll[6] = (self.alpha_w0 - 1) * self.p["w"]
        ll[7] = (self.beta_w0  - 1) * (1 - self.p["w"])
        
        ### p(phi)
        # phi_1
        ll[8]  = - 0.5 * log(2 * pi * self.lbd_phi0 * self.p["sigma2_1"]) * self.pdata
        ll[9]  = - 0.5 * phi_1_err / (self.lbd_phi0 * self.p["sigma2_1"])
        # phi_2
        ll[10] = - 0.5 * log(2 * pi * self.lbd_phi0 * self.p["sigma2_2"]) * self.pdata
        ll[11] = - 0.5 * phi_2_err / (self.lbd_phi0 * self.p["sigma2_2"])
        
        ### p(sigma2)
        # sigma2_1
        ll[12] = self.alpha_s20 * log(self.beta_s20)
        ll[13] = - gammaln(self.alpha_s20)
        ll[14] = - (self.alpha_s20 + 1.0) * log(self.p["sigma2_1"])
        ll[15] = - self.beta_s20 / self.p["sigma2_1"]
        # sigma2_2
        ll[16] = self.alpha_s20 * log(self.beta_s20)
        ll[17] = - gammaln(self.alpha_s20)
        ll[18] = - (self.alpha_s20 + 1.0) * log(self.p["sigma2_2"])
        ll[19] = - self.beta_s20 / self.p["sigma2_2"]
        
        return sum(ll), ll


    def EM_iter(self):
        """
            Executes a single round of EM updates for this model.

            Has checks to make sure that updates increase logl and
            that parameter values stay in sensible limits.
        """
        # self.ndata = T
        # self.pdata = P
        
        # phi1
#         sumxx          = self.X.T.dot(self.X)
#         sumxy          = self.X.T.dot(self.Y)
#         sigma_mu       = self.sigma_phi0_inv.dot(self.mu_phi0)
#         sigma_phi_inv  = self.sigma_phi0_inv + sumxx
#         self.p["phi1"] = solve(sigma_phi1_inv, sigma_mu + sumxy)
#         self.assert_logl_increased("phi1 update")

        # ==================== E-STEP ====================

        # ========== Responsibilities gamma ==========
        mean1 = float(item.dot(self.p["phi_1"]))
        std1  = sqrt(self.p["sigma2_1"])
        mean2 = float(item.dot(self.p["phi_2"]))
        std2  = sqrt(self.p["sigma2_2"])

        # data is 1 dimensional, and norm.pdf works on a vector returning probability for each separately
        propto_gamma1 = self.p["w"] * norm.pdf(self.X, mean1, std1)
        propto_gamma2 = (1 - self.p["w"]) * norm.pdf(self.X, mean2, std2)
        
        # elementwise, works because numpy
        gamma = propto_gamma1 / (propto_gamma1 + propto_gamma2)
        self.p["gamma"] = gamma
        sum_gamma = sum(gamma)
#         gamma2 = 1-gamma
        
#         gamma = zeros(self.ndata)
#         for t in range(self.ndata):
#             propto_gamma1 = self.p["w"] * norm.pdf(self.X[t], mean1, std1)
#             propto_gamma2 = (1 - self.p["w"]) * norm.pdf(self.X[t], mean2, std2)
#             gamma[t] = propto_gamma1 / (propto_gamma1 + propto_gamma2)

        # ==================== M-STEP ====================

        # ========== Weights w ==========
        num = sum_gamma + self.alpha_w0 - 1
        den = self.ndata - 2*sum_gamma - self.alpha_w0 - self.beta_w0 + 2
        self.p["w"] = 2 + (num / den)
        
        
        # ========== Variables phi ==========
        
#         sum_gammaxx   = sum(np.multiply(gamma, np.multiply(self.X, self.X)))
#         sum_gammaxy   = sum(np.multiply(gamma, np.multiply(self.X, self.Y)))
        sum_gammaxx   = np.dot(gamma, np.multiply(self.X, self.X))
        sum_gammaxy   = np.dot(gamma * self.Y, self.X)
        # ?????
        sigma_mu      = self.sigma_phi0_inv.dot(self.mu_phi0)
        sigma_phi_inv = self.sigma_phi0_inv + sumxx
        
        # phi_1
        
        
        # From linear model
#         sumxx         = self.X.T.dot(self.X)
#         sumxy         = self.X.T.dot(self.Y)
#         sigma_mu      = self.sigma_phi0_inv.dot(self.mu_phi0)
#         sigma_phi_inv = self.sigma_phi0_inv + sumxx
#         self.p["phi"] = solve(sigma_phi_inv, sigma_mu + sumxy)
#         self.assert_logl_increased("phi update")


        # ========== Variances sigma2 ==========
        
        # sigma2_1
        err  = (self.Y - self.X.dot(self.p["phi1"])) ** 2
        num  = self.beta_s20 + 0.5 * sum(gamma * err)
        den  = self.alpha_s20 + 1.0 + 0.5
        self.p["sigma2_1"] = num / den
        if self.p["sigma2_1"] < 0.0:
            raise ValueError("sigma2_1 < 0.0")
        
        # sigma2_2
        err  = (self.Y - self.X.dot(self.p["phi2"])) ** 2
        num  = self.beta_s20 + 0.5 * sum((1-gamma) * err)
        den  = self.alpha_s20 + 1.0 + 0.5
        self.p["sigma2_2"] = num / den
        if self.p["sigma2_2"] < 0.0:
            raise ValueError("sigma2_2 < 0.0")
        
        self.assert_logl_increased("sigma2 update")


        
#         # phi
#         sumxx         = self.X.T.dot(self.X)
#         sumxy         = self.X.T.dot(self.Y)
#         sigma_mu      = self.sigma_phi0_inv.dot(self.mu_phi0)
#         sigma_phi_inv = self.sigma_phi0_inv + sumxx
#         self.p["phi"] = solve(sigma_phi_inv, sigma_mu + sumxy)
#         self.assert_logl_increased("phi update")

#         # sigma2
#         phie = (self.p["phi"] - self.mu_phi0) ** 2
#         err  = (self.X.dot(self.p["phi"]) - self.Y) ** 2
#         num  = self.beta_s20 + 0.5 * sum(err) + 0.5 * sum(phie) / self.lbd_phi0
#         den  = self.alpha_s20 + 1.0 + 0.5 * (self.ndata + self.pdata)
#         self.p["sigma2"] = num / den
#         if self.p["sigma2"] < 0.0:
#             raise ValueError("sigma2 < 0.0")
#         self.assert_logl_increased("sigma2 update")

In [38]:
eye(12)

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

## Main

In [ ]:
# Aalto University, School of Science
# T-61.5140 Machine Learning: Advanced probabilistic Methods
# Author: antti.kangasraasio@aalto.fi, 2016

import matplotlib.pyplot as plt
from numpy import arange, min, max, sqrt, mean, std
from scipy.spatial.distance import cosine
import numpy as np

def main():
    """
        Executed when program is run.
    """
    print("Starting program")
    print("")
    test_LM_model()


def test_LM_model():
    """
        Example that demonstrates how to call the model.
    """
    # get hyperparameters for model
    hyperp = get_hyperp()
    # generate 50 training data and 20 validation data locations of dim=1
    ndata = 50
    ndata_v = 50
    pdata = 1
    X = generate_X(ndata, pdata)
    X_v = generate_X(ndata_v, pdata)
    # intialize true model randomly and draw observations from it
    true_model = EM_algo_LM(hyperp, ndata=ndata, pdata=pdata)
    Y, Z = generate_YZ(X, true_model)
    Y_v, Z_v = generate_YZ(X_v, true_model)
    print("Generated %d training data and %d validation data from true model:" % \
            (ndata, ndata_v))
    true_model.print_p()
    print("")

    # generate a model for estimating the parameters of the
    # true model based on the observations (X, Y) we just made
    model = EM_algo_LM(hyperp, X, Y)
    i, logl, r = model.EM_fit()
    print("Model fit (logl %.2f) after %d iterations (%s reached)" % \
            (logl, i, r))
    print("")
    print("MAP estimate of true model parameters:")
    model.print_p()
    print("")

    # crossvalidate the estimated model with the validation data
    fit_params = model.get_p()
    model_v = EM_algo_LM(hyperp, X_v, Y_v)
    model_v.set_p(fit_params)
    logl, ll = model_v.logl()
    print("Crossvalidated logl: %.2f" % (logl))

    # if possible, plot samples, true model and estimated model
    if pdata != 1:
        return
    plt.figure(figsize=(20,10))
    plt.scatter(X, Y, s=20, c='black', label="Training data")
    plt.scatter(X_v, Y_v, s=20, c='orange', label="Validation data")
    x = arange(min(X)-0.1, max(X)+0.1, 0.1)
    print_linear_model(x, true_model.get_p()["phi"], \
            true_model.get_p()["sigma2"], 'red', "True model")
    print_linear_model(x, model.get_p()["phi"], \
            model.get_p()["sigma2"], 'blue', "Predicted model")
    plt.legend(loc=1)
    plt.xlim(min(x), max(x))
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()
    


def print_linear_model(x, phi, sigma2, color, label):
    """
        Print linear model mean and 95% confidence interval.
    """
    y = phi * x
    plt.plot(x, y, color, label=label)
    plt.fill_between(x, y + 1.96 * sqrt(sigma2), y - 1.96 * sqrt(sigma2), \
            alpha=0.25, facecolor=color, interpolate=True)


if __name__ == "__main__":
    main()



In [52]:
X = generate_X(10, 2)

In [53]:
X.shape

(10, 2)

In [54]:
X

array([[ 1.22470801,  0.35364688],
       [-0.29730505,  0.86804445],
       [-0.51008941,  0.27406517],
       [-0.90501391,  1.43689847],
       [-0.16736504,  0.304238  ],
       [ 0.07960696,  0.17671287],
       [-1.37105112, -1.1726178 ],
       [ 0.67606752, -0.06235661],
       [ 0.54192738,  0.41945852],
       [-0.22212765, -0.15550222]])

In [55]:
X[8,:]

array([ 0.54192738,  0.41945852])

In [56]:
norm.pdf(X, 0, 1)

array([[ 0.1884555 ,  0.3747592 ],
       [ 0.3816949 ,  0.27370918],
       [ 0.3502759 ,  0.38423751],
       [ 0.26488391,  0.14209248],
       [ 0.39339383,  0.38089981],
       [ 0.39768018,  0.39276168],
       [ 0.15585501,  0.20059752],
       [ 0.31743818,  0.39816742],
       [ 0.34445867,  0.3653457 ],
       [ 0.38922065,  0.39414792]])

In [29]:
p_1 = np.array([0.3, 0.2])
p_2 = np.array([0.2, 0.8])
asd = p_1 / (p_1 + p_2)

In [30]:
asd

array([ 0.6,  0.2])

In [31]:
0.3 / (0.3+0.2)

0.6

In [32]:
0.2 / (0.2+0.8)

0.2

In [57]:
X.T.dot(X)

array([[ 5.38176416,  0.52539716],
       [ 0.52539716,  4.72119164]])

In [34]:
np.multiply(p_1, p_2)

array([ 0.06,  0.16])

In [58]:
true_model = EM_algo_LM(get_hyperp(), ndata=10, pdata=2)
Y, Z = generate_YZ(X, true_model)

In [59]:
Y.shape

(10,)

In [60]:
X.shape

(10, 2)

In [61]:
Z

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [62]:
X.T.dot(Y)

array([-1.92646461,  1.47475573])

In [50]:
?np.dot

In [63]:
X.T.dot(X)

array([[ 5.38176416,  0.52539716],
       [ 0.52539716,  4.72119164]])

In [81]:
eye(2) * np.sum(np.multiply(X, X),axis=0,keepdims=True)

array([[ 5.38176416,  0.        ],
       [ 0.        ,  4.72119164]])

In [102]:
X.T.dot(X)

array([[ 5.38176416,  0.52539716],
       [ 0.52539716,  4.72119164]])

In [101]:
np.dot(Y, np.multiply(X, X))

array([-0.04049531,  1.82697251])

In [84]:
total = np.array([0.0, 0.0])
for t in range(10):
    total += Y[t] * X[t] * X[t]

In [85]:
total

array([-0.04049531,  1.82697251])

In [98]:
asd1=Y * Y

In [99]:
asd1.shape

(10,)

In [100]:
np.dot(asd1, X)

array([-0.16473486,  0.96779041])

In [87]:
total2 = np.array([0.0, 0.0])
for t in range(10):
    total2 += Y[t] * X[t] * Y[t]

In [88]:
total2

array([-0.16473486,  0.96779041])

In [104]:
X.T.dot(X)

array([[ 5.38176416,  0.52539716],
       [ 0.52539716,  4.72119164]])